# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.


### 2) 
AFIRO        28 rows    32 cols        88 nonzeros       794 bytes      -4.6475314286E+02 optimalvalue

In [13]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader
using SolverTools
using SolverBenchmark

# Mehrotra formulation plus générale:

min $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $ Ax = b $ and $l \le x \le u$ 

In [14]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

function init_x0_lsq(A, b, lvar, uvar)
    x_tilde = A\b
    n = length(x_tilde)
    for i=1:n
        if x_tilde[i] <= lvar[i]
            x_tilde[i] = lvar[i] + 1
        elseif uvar[i] <= x_tilde[i]
            x_tilde[i] = uvar[i] - 1
        end
    end
    return x_tilde
end
    


init_x0_lsq (generic function with 1 method)

In [15]:


function Compute_Alpha_dual(v_k, dir_v_k)
    n = length(v_k)
    if n == 0
        return 0.
    end
    alphas = ones(n)
    for i=1:n
        if dir_v_k[i] < 0
            alphas[i] = -v_k[i] * 0.999 / dir_v_k[i]
        end
    end
    return minimum(alphas)
end


    
function Compute_Alpha_primal(v_k, dir_v_k, lvar, uvar)
    n = length(v_k)
    alpha_l, alpha_u = ones(n), ones(n)
    for i=1:n
        if dir_v_k[i] > 0
            alpha_u[i] = (uvar[i] - v_k[i]) * 0.999 / dir_v_k[i]
        elseif dir_v_k[i] < 0
            alpha_l[i] = (lvar[i] - v_k[i]) * 0.999 / dir_v_k[i]
        end
    end
    return min(minimum(alpha_l), minimum(alpha_u))
end

function Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    #x_l coordinates of x corresponding to finite lower bounds ( resp. finite upper bounds for x_u)
    # arguments must have finite bounds 
    return (s_l' * (x_l-lvar) + s_u' * (uvar-x_u)) / (length(x_l) + length(x_u))
end


function is_in_Neighborhood_inf(gamma, x_l, x_u, s_l, s_u, lvar, uvar)
    # check if the current point is in N_inf(gamma)
    # true : (xi_l - lvari) * si_l >= gamma mu   and   (uvari - xi_u) * si_u >= gamma mu 
    mu = Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    for i=1:length(x_l)
        if (x_l[i] - lvar[i]) * s_l[i] < gamma*mu
            return false
        end
    end
    for i=1:length(x_u)
        if (uvar[i] - x_u[i]) * s_u[i] < gamma*mu
            return false
        end
    end
    return true
end

is_in_Neighborhood_inf (generic function with 1 method)

In [97]:

function MehrotraPCQuadBounds(QM, max_iter; eps=1e-10, tol_step_x=1e-8, eps_mu=1e-10, display=true)
    
    start_time = time()
    elapsed_time = 0.0
    
    # get variables from QuadraticModel
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    x_k = init_x0(lvar, uvar)
    id_non_inf_l, id_non_inf_u = findall((x -> x!=-Inf), lvar), findall((x -> x!=Inf), uvar) # finite bounds index
    @assert all(x_k .> lvar) && all(x_k .< uvar)
    A = jac(QM, x_k)
    n_rows, n_cols = size(A) 
    Q = hess(QM, x_k)
    c = grad(QM, zeros(n_cols))
    c0 = obj(QM, zeros(n_cols))
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    x_k = init_x0_lsq(A, b, lvar, uvar)
    @assert all(x_k .> lvar) && all(x_k .< uvar)
    s_k_l, s_k_u = zeros(n_cols), zeros(n_cols)
    s_k_l[id_non_inf_l] .= 1.
    s_k_u[id_non_inf_u] .= 1.
    lambda_k = Matrix(A)'\(c +Q*x_k) # least square initialisation, s_0 = stilde_0
    
    rb_k = A * x_k - b
    rc_k = -Q * x_k + A' * lambda_k + s_k_l - s_k_u - c
    mu_k = Compute_mu(x_k[id_non_inf_l], x_k[id_non_inf_u], 
                        s_k_l[id_non_inf_l], s_k_u[id_non_inf_u], 
                        lvar[id_non_inf_l], uvar[id_non_inf_u])

    k = 0
    e = ones(n_cols)
    n_c = norm(c)
    n_b = norm(b)
    
    # matrices without infinity constraints
    nb_non_inf_l, nb_non_inf_u = length(id_non_inf_l), length(id_non_inf_u) # number of finite constraints
    
    Xk_non_inf_l = Diagonal(x_k[id_non_inf_l])
    Sk_non_inf_l = spzeros(nb_non_inf_l, n_cols)
    I_non_inf_l = spzeros(n_cols, nb_non_inf_l)
    for j=1:nb_non_inf_l   # cartesian index not working when nb_non_inf_l = 0
        Sk_non_inf_l[j, id_non_inf_l[j]] = s_k_l[id_non_inf_l[j]]
        I_non_inf_l[id_non_inf_l[j], j] = 1
    end
    
    Xk_non_inf_u = Diagonal(x_k[id_non_inf_u])
    Sk_non_inf_u = spzeros(nb_non_inf_u, n_cols)
    I_op_non_inf_u = spzeros(n_cols, nb_non_inf_u)
    for j=1:nb_non_inf_u 
        Sk_non_inf_u[j, id_non_inf_u[j]] = s_k_u[id_non_inf_u[j]]
        I_op_non_inf_u[id_non_inf_u[j], j] = -1
    end
    Lvar_non_inf = Diagonal(lvar[id_non_inf_l]) 
    Uvar_non_inf = Diagonal(uvar[id_non_inf_u]) 

    # stopping criterion
    quad_part = x_k' * Q * x_k 
    pdd = abs(quad_part + c' * x_k - b' * lambda_k ) / (1 + abs(c' * x_k + quad_part/2))
    cond_rb, cond_rc = norm(rb_k) / (1 + n_b), norm(rc_k) / (1 + n_c)
    opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps # start false?
    small_step_x, small_mu = false, mu_k < eps_mu

    # display
    if display == true
        println("Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu")
        println("--------------------------------------------------------------------------------------------------------")
        @printf("% 4d |    % 7.4e   |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,0., mu_k)
    end
    

    while k<max_iter && (!opti_pdd || !opti_rb || !opti_rc) && !small_step_x && !small_mu
        
        
        # Affine scaling direction
        Jacob_Fk = [-Q            A'                             I_non_inf_l                         I_op_non_inf_u
                    A             spzeros(n_rows, n_rows)        spzeros(n_rows, nb_non_inf_l)       spzeros(n_rows, nb_non_inf_u)
                    Sk_non_inf_l  spzeros(nb_non_inf_l, n_rows)  Xk_non_inf_l-Lvar_non_inf           spzeros(nb_non_inf_l, nb_non_inf_u)
                    Sk_non_inf_u  spzeros(nb_non_inf_u, n_rows)  spzeros(nb_non_inf_u,nb_non_inf_l)  Xk_non_inf_u-Uvar_non_inf]
        Fk_aff = [-rc_k
                  -rb_k
                  -(x_k[id_non_inf_l]-lvar[id_non_inf_l]).*s_k_l[id_non_inf_l]
                  -(x_k[id_non_inf_u]-uvar[id_non_inf_u]).*s_k_u[id_non_inf_u]]

        dir_aff_k = Jacob_Fk\Fk_aff
        
        alpha_aff_pri = Compute_Alpha_primal(x_k, dir_aff_k[1:n_cols], lvar, uvar)
        alpha_aff_dual_l = Compute_Alpha_dual(s_k_l[id_non_inf_l], 
                                            dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l])
        alpha_aff_dual_u = Compute_Alpha_dual(s_k_u[id_non_inf_u], 
                                                dir_aff_k[n_rows+n_cols+nb_non_inf_l+1:end])
        # pb above compute alpha dual

        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        if nb_non_inf_l == 0
            alpha_aff_dual_final = alpha_aff_dual_u
        elseif nb_non_inf_u == 0
            alpha_aff_dual_final = alpha_aff_dual_l
        else
            alpha_aff_dual_final = min(alpha_aff_dual_l, alpha_aff_dual_u)
        end
        
        mu_aff = Compute_mu(x_k[id_non_inf_l] + alpha_aff_pri * dir_aff_k[1:n_cols][id_non_inf_l],
                    x_k[id_non_inf_u] + alpha_aff_pri * dir_aff_k[1:n_cols][id_non_inf_u],
                    s_k_l[id_non_inf_l] + alpha_aff_dual_final * dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l],
                    s_k_u[id_non_inf_u] + alpha_aff_dual_final * dir_aff_k[n_rows+n_cols+nb_non_inf_l+1: end],
                    lvar[id_non_inf_l], uvar[id_non_inf_u])
        
        sigma = (mu_aff / mu_k)^3

        # corrector and centering step
        Fk_cc = [zeros(n_rows+n_cols, 1)
                 sigma*mu_k*e[1:nb_non_inf_l] - dir_aff_k[1:n_cols][id_non_inf_l].*dir_aff_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l]
                 -sigma*mu_k*e[1:nb_non_inf_u] - dir_aff_k[1:n_cols][id_non_inf_u].*dir_aff_k[n_rows+n_cols+nb_non_inf_l+1: end]] 

        dir_cc_k = Jacob_Fk\Fk_cc
        dir_k = dir_aff_k + dir_cc_k # final direction
        
        alpha_k_pri = Compute_Alpha_primal(x_k, dir_k[1:n_cols], lvar, uvar)
        alpha_k_dual_l = Compute_Alpha_dual(s_k_l[id_non_inf_l], 
                                        dir_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l])
        
        alpha_k_dual_u = Compute_Alpha_dual(s_k_u[id_non_inf_u], 
                                            dir_k[n_rows+n_cols+nb_non_inf_l+1: end])
        
        if nb_non_inf_l == 0
            alpha_dual_final = alpha_k_dual_u
        elseif nb_non_inf_u == 0
            alpha_dual_final = alpha_k_dual_l
        else
            alpha_dual_final = min(alpha_k_dual_l, alpha_k_dual_u)
        end
        
        # new parameters
        x_k += alpha_k_pri * dir_k[1:n_cols]
        lambda_k += alpha_dual_final * dir_k[n_cols+1: n_rows+n_cols]
        s_k_l[id_non_inf_l] += alpha_dual_final * dir_k[n_rows+n_cols+1: n_rows+n_cols+nb_non_inf_l]
        s_k_u[id_non_inf_u] += alpha_dual_final * dir_k[n_rows+n_cols+nb_non_inf_l+1: end]
        
        Xk_non_inf_l = Diagonal(x_k[id_non_inf_l])
        for j=1:nb_non_inf_l
            Sk_non_inf_l[j, id_non_inf_l[j]] = s_k_l[id_non_inf_l[j]]
        end
        Xk_non_inf_u = Diagonal(x_k[id_non_inf_u])
        
        for j=1:nb_non_inf_u
            Sk_non_inf_u[j, id_non_inf_u[j]] = s_k_u[id_non_inf_u[j]]
        end
        
        step_x = norm(alpha_k_pri * dir_k[1:n_cols])
        mu_k = Compute_mu(x_k[id_non_inf_l], x_k[id_non_inf_u], 
                        s_k_l[id_non_inf_l], s_k_u[id_non_inf_u], 
                        lvar[id_non_inf_l], uvar[id_non_inf_u])

        rb_k = A * x_k - b
        rc_k = -Q * x_k + A' * lambda_k + s_k_l - s_k_u - c
        
        # update stopping criterion values:
        quad_part = x_k' * Q * x_k 
        pdd = abs(quad_part + c' * x_k - b' * lambda_k - s_k_l[id_non_inf_l]'*lvar[id_non_inf_l] +  s_k_u[id_non_inf_u]'*uvar[id_non_inf_u] ) / 
                    (1 + abs(c' * x_k + quad_part/2)) 
        cond_rb = norm(rb_k) / (1 + n_b)
        cond_rc = norm(rc_k) / (1 + n_c)
        opti_pdd, opti_rb, opti_rc = pdd < eps, cond_rb < eps, cond_rc < eps
        small_step_x, small_mu = step_x < tol_step_x, mu_k < eps_mu
        k += 1
        
        if display == true
            @printf("% 4d |    % 7.4e   |        % 7.2e       |   % 7.2e  |   % 7.2e  | % 7.2e  | % 7.2e\n", 
                k, c0+c'*x_k +quad_part/2, pdd, cond_rb, cond_rc,step_x, mu_k)
        end
        

    end
    
    if display == true
        criteria = [k >= max_iter,  opti_pdd, opti_rb, opti_rc, small_step_x, small_mu]
        criteria_names = ["reached max_iter",  "pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", 
            "step_x <= small_step_x", "mu_k <= eps_mu"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    
    elapsed_time = time() - start_time
    
    GenExStats = GenericExecutionStats(:acceptable, QM, solution = x_k,
                               objective = c0 + c'*x_k + x_k'*Q*x_k/2 , 
                                dual_feas = norm(rc_k), primal_feas = norm(rb_k),
                                multipliers = lambda_k,
                                iter = k, elapsed_time=elapsed_time)
    println(GenExStats)
    
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_l_opt" => s_k_l, "s_u_opt" => s_k_u, 
        "n_iter" => k, "pdd" => pdd, "cond_rb" => cond_rb, "cond_rc" => cond_rc)
end

MehrotraPCQuadBounds (generic function with 1 method)

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [98]:
# probleme1
Q = [6 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
c0 = 0.
A = [1 0 1
    0 1 1]
b = [0; 3]
lvar = [0;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];


In [99]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [100]:
res_mpc1 =  MehrotraPCQuadBounds(SM, 20);
display_results(res_mpc1)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     1.4000e+01   |         6.67e-01       |    5.59e-01  |    2.37e-01  |  0.00e+00  |  1.67e+00
   1 |     1.2894e+01   |         6.89e-03       |    2.96e-02  |    4.17e-02  |  1.34e+00  |  3.25e-02
   2 |     1.3501e+01   |         1.31e-03       |    6.02e-04  |    8.48e-04  |  1.18e-01  |  2.00e-03
   3 |     1.3500e+01   |         1.23e-06       |    6.26e-07  |    8.30e-07  |  2.27e-03  |  2.03e-06
   4 |     1.3500e+01   |         1.23e-09       |    6.26e-10  |    8.30e-10  |  2.25e-06  |  2.03e-09
   5 |     1.3500e+01   |         1.23e-12       |    6.26e-13  |    8.30e-13  |  2.25e-09  |  2.03e-12

 stopping criterion = ["pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", "step_x <= small_step_x", "mu_k <= eps_mu"]


MethodError: MethodError: no method matching GenericExecutionStats(::Symbol, ::QuadraticModel; solution=[2.047991330173787e-12, 3.0000000000009277, 1.9238794341633236e-13], objective=13.50000000000089, dual_feas=8.34885512966579e-12, primal_feas=2.5047322373864897e-12, multipliers=[-9.943359232775876, 12.000000000013133], iter=5, elapsed_time=0.2690000534057617)
Closest candidates are:
  GenericExecutionStats(::Symbol, ::AbstractNLPModel; solution, objective, dual_feas, primal_feas, iter, elapsed_time, solver_specific) at C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\qgaY0\src\stats\stats.jl:78 got unsupported keyword argument "multipliers"
  GenericExecutionStats(::Any, ::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any) at C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\qgaY0\src\stats\stats.jl:58 got unsupported keyword arguments "solution", "objective", "dual_feas", "primal_feas", "multipliers", "iter", "elap

In [26]:
#pb verif

QM_verif = QuadraticModel(c*(-1), Q, A=A*(-1), lcon=lcon, ucon=ucon, lvar=-uvar, uvar=lvar, x0=x01, c0=c0, name="QM1")
SM_verif = SlackModel(QM_verif)
res_mpc_verif =  MehrotraPCQuadBounds(SM_verif, 20);
display_results(res_mpc_verif)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     1.4000e+01   |         6.67e-01       |    5.59e-01  |    2.37e-01  |  0.00e+00  |  1.67e+00
   1 |     1.2894e+01   |         6.89e-03       |    2.96e-02  |    4.17e-02  |  1.34e+00  |  3.25e-02
   2 |     1.3501e+01   |         1.31e-03       |    6.02e-04  |    8.48e-04  |  1.18e-01  |  2.00e-03
   3 |     1.3500e+01   |         1.23e-06       |    6.26e-07  |    8.30e-07  |  2.27e-03  |  2.03e-06
   4 |     1.3500e+01   |         1.23e-09       |    6.26e-10  |    8.30e-10  |  2.25e-06  |  2.03e-09
   5 |     1.3500e+01   |         1.23e-12       |    6.26e-13  |    8.30e-13  |  2.25e-09  |  2.03e-12

 stopping criterion = ["pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", "step_x <= small_step_x", "mu_k <= eps_mu"]

---------------------------------------------------

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [-2.04799e-12, -3.0, -1.92388e-13]
  "lambda_opt" => [-9.94336, 12.0]
  "s_l_opt"    => [0.0, 0.0, 0.0]
  "s_u_opt"    => [1.94336, 6.38435e-13, 0.943359]
  "n_iter"     => 5
  "pdd"        => 1.23291e-12
  "cond_rb"    => 6.26183e-13
  "cond_rc"    => 8.30287e-13

In [27]:
### pb verif 2
c2 = [1; 0; 0]
A2 = Matrix([1, 1, 1]')
b2 = [1];
QM_verif2 = QuadraticModel(c2, zeros(3,3), A=A2, lcon=b2, ucon=b2, lvar=[0.;0;0], 
                uvar=[Inf; Inf; Inf], x0=x01, c0=c0, name="QMverfi2")
SM_verif2 = SlackModel(QM_verif2)
res_mpc_verif2 =  MehrotraPCQuadBounds(SM_verif2, 20);
display_results(res_mpc_verif2)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     1.0000e+00   |         3.33e-01       |    1.00e+00  |    9.57e-01  |  0.00e+00  |  1.00e+00
   1 |     1.1111e-01   |         4.04e-01       |    0.00e+00  |    1.84e-16  |  1.19e+00  |  1.50e-01
   2 |     6.4513e-03   |         6.75e-03       |    0.00e+00  |    5.55e-17  |  1.28e-01  |  2.26e-03
   3 |     6.4513e-06   |         6.79e-06       |    0.00e+00  |    3.83e-20  |  7.89e-03  |  2.26e-06
   4 |     6.4513e-09   |         6.79e-09       |    0.00e+00  |    1.11e-16  |  7.89e-06  |  2.26e-09
   5 |     6.4513e-12   |         6.79e-12       |    0.00e+00  |    2.58e-26  |  7.89e-09  |  2.26e-12

 stopping criterion = ["pdd <= eps", "cond_rb <= eps", "cond_rc <= eps", "step_x <= small_step_x", "mu_k <= eps_mu"]

---------------------------------------------------

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [6.45129e-12, 0.5, 0.5]
  "lambda_opt" => [-3.38005e-13]
  "s_l_opt"    => [1.0, 3.38005e-13, 3.38005e-13]
  "s_u_opt"    => [0.0, 0.0, 0.0]
  "n_iter"     => 5
  "pdd"        => 6.7893e-12
  "cond_rb"    => 0.0
  "cond_rc"    => 2.58494e-26

# Lecture des donnees .SIF

In [28]:

function createQuadraticModel(qpdata, name_pb)
    # probleme du point initial
    x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0, x0=x0, name=name_pb)
end

function displayQuadraticModel(QM)
    #println("A = ", Matrix(jac(QM, QM.meta.x0)))
    #println("Q = ", Matrix(hess(QM, QM.meta.x0)))
    println("lvar = ", QM.meta.lvar)
    println("uvar = ", QM.meta.uvar)
    println("x0 = ", QM.meta.x0)
    #println("lcon = ", QM.meta.lcon)
    #println("ucon = ", QM.meta.ucon)
end

displayQuadraticModel (generic function with 1 method)

In [29]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib\\AFIRO.SIF"

In [30]:
qpdata2 = readqps(afiro)  # obj   -4.6475314286E+02
qpdata2.contypes
QM2 = createQuadraticModel(qpdata2, "LP2")
SM2 = SlackModel(QM2);
#displayQuadraticModel(SM2)

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [31]:
res_mpc2 =  MehrotraPCQuadBounds(SM2, 1000, tol_step_x = 1e-10);
display_results(res_mpc2)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     4.3820e+02   |         6.83e-01       |    4.44e-01  |    9.54e-01  |  0.00e+00  |  3.64e+01
   1 |     3.4744e+02   |         5.55e+00       |    4.22e-01  |    9.22e-01  |  1.45e+01  |  3.61e+01
   2 |    -5.4715e+00   |         2.38e+02       |    3.85e-01  |    8.58e-01  |  6.16e+01  |  3.54e+01
   3 |     3.6743e+01   |         3.61e+01       |    3.84e-16  |    7.04e-01  |  2.20e+02  |  3.31e+01
   4 |     6.7364e+01   |         1.07e+01       |    4.28e-16  |    3.31e-01  |  4.45e+01  |  1.76e+01
   5 |    -1.5940e+02   |         1.70e+00       |    5.84e-16  |    1.83e-01  |  3.13e+02  |  9.31e+00
   6 |    -4.0241e+02   |         3.17e-01       |    2.29e-15  |    4.03e-02  |  4.98e+02  |  3.99e+00
   7 |    -4.5934e+02   |         4.20e-02       |    1.03e-15  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [80.0, 25.5, 54.5, 84.8, 46.7321, 3.34594e-11, 3.31488e-11, 3…
  "lambda_opt" => [-0.628571, 2.18196e-11, -0.344771, -0.228571, -1.30837e-12, …
  "s_l_opt"    => [7.57324e-13, 5.26156e-12, 6.86417e-13, 7.14501e-13, 1.25103e…
  "s_u_opt"    => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.28067…
  "n_iter"     => 11
  "pdd"        => 6.39303e-12
  "cond_rb"    => 2.19009e-15
  "cond_rc"    => 2.56398e-13

In [32]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\KB2.SIF")
qpdata3 = readqps(kb2)
QM3 = createQuadraticModel(qpdata3, "QM3")
SM3 = SlackModel(QM3);

┌ Info: Using 'KB2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'FAT7..J.' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '77BOUND' as BOUNDS (l. 227)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


In [33]:
res_mpc3 =  MehrotraPCQuadBounds(SM3, 20, tol_step_x = 1e-12);
display_results(res_mpc3)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |     1.1675e+01   |         9.21e-01       |    2.49e+03  |    4.63e-01  |  0.00e+00  |  6.18e+00
   1 |     2.4577e+01   |         1.05e+01       |    2.08e+03  |    2.52e-01  |  2.93e+00  |  4.63e+00
   2 |     9.7097e-01   |         5.71e+01       |    4.40e+02  |    1.11e-01  |  6.36e+00  |  1.86e+00
   3 |    -6.3645e+00   |         9.90e+00       |    2.04e+02  |    7.76e-02  |  2.74e+01  |  1.37e+00
   4 |    -3.0768e+01   |         7.38e-01       |    7.82e+01  |    5.18e-02  |  1.26e+02  |  1.04e+00
   5 |    -3.3028e+02   |         8.64e-01       |    5.41e-13  |    4.01e-02  |  1.54e+03  |  1.47e+00
   6 |    -5.2433e+02   |         8.69e-01       |    7.05e-13  |    3.82e-02  |  9.83e+02  |  1.80e+00
   7 |    -6.2643e+02   |         8.27e-01       |    1.03e-12  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [0.909072, 0.000270472, 0.0121787, 0.00829969, 4.59991, 26.98…
  "lambda_opt" => [17.2763, 17.211, 16.9402, 16.6568, 11.9999, 17.2139, 17.431,…
  "s_l_opt"    => [0.00251664, 0.0583318, 0.0450346, 0.099087, 0.000546074, 0.0…
  "s_u_opt"    => [0.0, 0.000260407, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  … …
  "n_iter"     => 20
  "pdd"        => 0.00010617
  "cond_rb"    => 2.33524e-12
  "cond_rc"    => 3.68976e-16

In [34]:
# problem 4 SC50A  obj  -6.4575077059E+01
SC50A = string(path_pb, "\\SC50A.SIF")
qpdata4 = readqps(SC50A)
QM4 = createQuadraticModel(qpdata4, "QM4")
SM4 = SlackModel(QM4);

┌ Info: Using 'SC50A' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'MAXIM' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'CONST' as RHS (l. 153)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [35]:
res_mpc4 =  MehrotraPCQuadBounds(SM4, 20, tol_step_x = 1e-12);
display_results(res_mpc4)

Iter | primal_objective | primal-dual difference | rb condition | rc condition |   step x   |     mu
--------------------------------------------------------------------------------------------------------
   0 |    -1.0000e+00   |         5.00e-01       |    1.30e+01  |    4.42e+00  |  0.00e+00  |  2.01e+01
   1 |    -8.1774e-01   |         5.84e+02       |    1.30e-02  |    3.06e+00  |  3.55e+01  |  1.45e+01
   2 |    -8.9022e-01   |         2.36e+01       |    1.30e-05  |    1.30e-01  |  7.66e-01  |  6.12e-01
   3 |    -9.3339e+00   |         3.42e+00       |    1.04e-05  |    1.23e-01  |  1.11e+02  |  5.93e-01
   4 |    -3.2552e+01   |         4.92e-01       |    7.80e-06  |    7.91e-02  |  3.01e+02  |  4.57e-01
   5 |    -5.4487e+01   |         1.03e-02       |    2.91e-06  |    6.01e-02  |  2.99e+02  |  3.24e-01
   6 |    -6.0759e+01   |         2.04e-02       |    8.31e-07  |    2.89e-02  |  1.03e+02  |  1.96e-01
   7 |    -6.3542e+01   |         1.94e-02       |    1.64e-07  | 

OrderedDict{String,Any} with 8 entries:
  "x_opt"      => [1.76119e-11, 16.5687, 64.5751, 64.5751, 64.5751, 1.76119e-11…
  "lambda_opt" => [-4.28388e-15, -0.138705, -513941.0, -0.208058, -0.124759, -0…
  "s_l_opt"    => [0.013946, 1.49751e-14, 3.7416e-15, 3.74898e-15, 3.74898e-15,…
  "s_u_opt"    => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.02311…
  "n_iter"     => 12
  "pdd"        => 3.53456e-13
  "cond_rb"    => 9.53975e-14
  "cond_rc"    => 5.20628e-17

In [36]:
T = typeof(SM3.meta)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname nvar is Int64
type of the fieldname x0 is Array{T,1} where T
type of the fieldname lvar is Array{T,1} where T
type of the fieldname uvar is Array{T,1} where T
type of the fieldname ifix is Array{Int64,1}
type of the fieldname ilow is Array{Int64,1}
type of the fieldname iupp is Array{Int64,1}
type of the fieldname irng is Array{Int64,1}
type of the fieldname ifree is Array{Int64,1}
type of the fieldname iinf is Array{Int64,1}
type of the fieldname nbv is Int64
type of the fieldname niv is Int64
type of the fieldname nlvb is Int64
type of the fieldname nlvo is Int64
type of the fieldname nlvc is Int64
type of the fieldname nlvbi is Int64
type of the fieldname nlvci is Int64
type of the fieldname nlvoi is Int64
type of the fieldname nwv is Int64
type of the fieldname ncon is Int64
type of the fieldname y0 is Array{T,1} where T
type of the fieldname lcon is Array{T,1} where T
type of the fieldname ucon is Array{T,1} where T
type of the fieldname jfix is Array{Int64,1

In [37]:
typeof(SM3.model) == QuadraticModel

true